In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error as mae
import numpy as np
data = pd.read_csv('nonlinear_rg.csv')
data.drop('date', axis = 1, inplace = True)

np.random.seed(7) #для воспроизводимости результатов

X = data.loc[:, 'a':'b']
Y = data['y']
X_train, Y_train = X[:-120], Y[:-120]
X_test, Y_test = X[-120:], Y[-120:]
X_train['exp(a)'] = np.exp(X_train['a'])
X_train['sqrt(b)'] = np.sqrt(abs(X_train['b']))
X_test['exp(a)'] = np.exp(X_test['a'])
X_test['sqrt(b)'] = np.sqrt(abs(X_test['b']))
scale = MinMaxScaler(feature_range=(0, 1))
X_train = scale.fit_transform(X_train)
X_test = scale.fit_transform(X_test)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.htm

In [1]:
from keras.models import Sequential, Model
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Dense
from keras import regularizers

Using TensorFlow backend.


In [ ]:
def reg_model():
    model = Sequential()
    model.add(Dense(5, input_dim = 4, kernel_initializer = 'random_normal', activation = 'relu'))
    model.add(Dense(20,  kernel_initializer = 'random_normal', activation = 'linear', 
                    kernel_regularizer = regularizers.l2(0.01),
                activity_regularizer = regularizers.l1(0.01),
                   bias_regularizer = regularizers.l1_l2(0.01)
                   ))
    model.add(Dense(12,  kernel_initializer = 'random_normal', activation = 'linear'))
    model.add(Dense(1,  kernel_initializer = 'random_normal', activation = 'linear'))
    model.compile(loss = 'mean_absolute_error', optimizer = 'sgd')
    return model

model = KerasRegressor(build_fn = reg_model, epochs = 10, batch_size = 1,verbose=1)
model.fit(X_train,Y_train)


print('точность: ', mae(model.predict(X_test), Y_test))

Epoch 1/10
1706/1706 [==============================] - 7s - loss: 0.9853     
Epoch 2/10
1706/1706 [==============================] - 3s - loss: 0.9778     
Epoch 3/10
 648/1706 [==========>...................] - ETA: 2s - loss: 0.9006

In [3]:
model = Sequential()
model.add(Dense(5, input_dim = 4, kernel_initializer = 'random_normal', activation = 'relu', name = 'layer1'))
model.add(Dense(20,  kernel_initializer = 'random_normal', activation = 'linear', name = 'layer2'))
model.add(Dense(12,  kernel_initializer = 'random_normal', activation = 'linear', name = 'layer3'))
model.add(Dense(1,  kernel_initializer = 'random_normal', activation = 'linear', name = 'layer4'))
model.compile(loss = 'mean_absolute_error', optimizer = 'sgd')
model.fit(X_train, Y_train, epochs = 30, batch_size = 1)
print('точность: ', mae(model.predict(X_test), Y_test))
model.summary()

Epoch 1/30
1706/1706 [==============================] - 3s - loss: 0.9829     
Epoch 2/30
1706/1706 [==============================] - 2s - loss: 0.9766     
Epoch 3/30
1706/1706 [==============================] - 2s - loss: 0.9757     
Epoch 4/30
1706/1706 [==============================] - 2s - loss: 0.9765     
Epoch 5/30
1706/1706 [==============================] - 2s - loss: 0.9749     
Epoch 6/30
1706/1706 [==============================] - 2s - loss: 0.9767     
Epoch 7/30
1706/1706 [==============================] - 2s - loss: 0.9762     
Epoch 8/30
1706/1706 [==============================] - 2s - loss: 0.9731     
Epoch 9/30
1706/1706 [==============================] - 2s - loss: 0.9759     
Epoch 10/30
1706/1706 [==============================] - 2s - loss: 0.9747     
Epoch 11/30
1706/1706 [==============================] - 2s - loss: 0.9751     
Epoch 12/30
1706/1706 [==============================] - 2s - loss: 0.9773     
Epoch 13/30
1706/1706 [==========================

In [4]:
# значение выхода layer3 для элемента X_test[1]
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.output_shape)
    
mod = Model(input = model.input, output = model.get_layer('layer3').output)
mod.predict(X_test[0].reshape((-1, 4)))

0 layer1 (None, 5)
1 layer2 (None, 20)
2 layer3 (None, 12)
3 layer4 (None, 1)


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("la..., outputs=Tensor("la...)`
  """


array([[-0.32237804,  0.15223962,  0.28404945, -0.03054404, -0.27396214,
         0.20735724, -0.08910358,  0.22887455,  0.1134897 , -0.12192211,
        -0.39964908, -0.19414499]], dtype=float32)

In [19]:
#модель с 2-мя входами 3-мя выходами

from keras.layers import Input, concatenate
from keras.models import Model

input_1 = Input(shape = (2, ) , dtype = 'float32', name='input_1')
layer_1 = Dense(3, kernel_initializer = 'random_normal', activation = 'relu')(input_1)
layer_1 = Dense(20,  kernel_initializer = 'random_normal', activation = 'linear')(layer_1)
output_1 = Dense(1, activation='linear', name = 'output_1')(layer_1)


input_2 = Input(shape = (2, ), dtype = 'float32', name = 'input_2')
layer_2 = Dense(3, kernel_initializer = 'random_normal', activation = 'relu')(input_2)
layer_2 = Dense(20,  kernel_initializer = 'random_normal', activation = 'linear')(layer_2)
output_2 = Dense(1, activation='linear', name = 'output_2')(layer_2)

x = concatenate([layer_1, layer_2])
x = Dense(15, activation='linear')(x)
x = Dense(1,  kernel_initializer = 'random_normal', activation = 'linear')(x)

model = Model(inputs=[input_1, input_2], outputs = [x, output_1, output_2])

model.compile(loss = 'mean_absolute_error', optimizer = 'sgd', loss_weights=[1., .3, .2])

model.fit([X_train[:, [0, 2]], X_train[:, [1, 3]]], [Y_train, Y_train, Y_train], epochs = 20, batch_size = 1)

print('точность: ', mae(np.array(model.predict([X_test[:, [0, 2]], X_test[:, [1, 3]]]))[2], Y_test))

Epoch 1/20
1706/1706 [==============================] - 4s - loss: 1.4689 - dense_36_loss: 0.9800 - output_1_loss: 0.9745 - output_2_loss: 0.9826     
Epoch 2/20
1706/1706 [==============================] - 3s - loss: 1.1821 - dense_36_loss: 0.7705 - output_1_loss: 0.7266 - output_2_loss: 0.9679     
Epoch 3/20
1706/1706 [==============================] - 4s - loss: 0.7918 - dense_36_loss: 0.4422 - output_1_loss: 0.5594 - output_2_loss: 0.9091     
Epoch 4/20
1706/1706 [==============================] - 3s - loss: 0.7697 - dense_36_loss: 0.4258 - output_1_loss: 0.5587 - output_2_loss: 0.8816     
Epoch 5/20
1706/1706 [==============================] - 3s - loss: 0.7623 - dense_36_loss: 0.4187 - output_1_loss: 0.5572 - output_2_loss: 0.8823     
Epoch 6/20
1706/1706 [==============================] - 4s - loss: 0.7664 - dense_36_loss: 0.4228 - output_1_loss: 0.5576 - output_2_loss: 0.8817     
Epoch 7/20
1706/1706 [==============================] - 4s - loss: 0.7610 - dense_36_loss: 0.4

In [14]:
from keras.layers import LSTM
X_train = np.reshape(X_train, (len(X_train), 4, 1))
X_test = np.reshape(X_test, (len(X_test), 4, 1))
model = Sequential()
model.add(LSTM(3, input_shape=(X_train.shape[1], X_train.shape[2]), kernel_initializer = 'random_normal'))
model.add(Dense(12,  kernel_initializer = 'random_normal', activation = 'linear'))
model.add(Dense(1,  kernel_initializer = 'random_normal', activation = 'linear'))
model.compile(loss = 'mean_absolute_error', optimizer = 'sgd')
model.fit(X_train, Y_train, epochs = 20, batch_size = 1)
print('точность: ', mae(model.predict(X_test), Y_test))

Epoch 1/20
1706/1706 [==============================] - 13s - loss: 0.9854    
Epoch 2/20
1706/1706 [==============================] - 12s - loss: 0.9736    
Epoch 3/20
1706/1706 [==============================] - 12s - loss: 0.9728    
Epoch 4/20
1706/1706 [==============================] - 12s - loss: 0.9645    
Epoch 5/20
1706/1706 [==============================] - 13s - loss: 0.7511    
Epoch 6/20
1706/1706 [==============================] - 13s - loss: 0.5515    
Epoch 7/20
1706/1706 [==============================] - 13s - loss: 0.5207    
Epoch 8/20
1706/1706 [==============================] - 13s - loss: 0.5041    
Epoch 9/20
1706/1706 [==============================] - 13s - loss: 0.4878    
Epoch 10/20
1706/1706 [==============================] - 13s - loss: 0.4750    
Epoch 11/20
1706/1706 [==============================] - 13s - loss: 0.4653    
Epoch 12/20
1706/1706 [==============================] - 13s - loss: 0.4501    
Epoch 13/20
1706/1706 [==========================

In [49]:
from keras.layers import  Conv1D, GlobalMaxPooling1D
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
model = Sequential()
model.add(Conv1D(5, kernel_size = 2, activation = 'relu', input_shape = (4, 1), padding = 'same'))
model.add(GlobalMaxPooling1D())
model.add(Dense(12,  kernel_initializer = 'random_normal', activation = 'linear', name = 'layer3'))
model.add(Dense(1,  kernel_initializer = 'random_normal', activation = 'linear', name = 'layer4'))
model.compile(loss = 'mean_absolute_error', optimizer = 'sgd')
model.fit(X_train, Y_train, epochs = 30, batch_size = 1)
print('точность: ', mae(model.predict(X_test), Y_test))

Epoch 1/30
1706/1706 [==============================] - 4s - loss: 0.8737     
Epoch 2/30
1706/1706 [==============================] - 2s - loss: 0.6407     
Epoch 3/30
1706/1706 [==============================] - 2s - loss: 0.5446     
Epoch 4/30
1706/1706 [==============================] - 2s - loss: 0.5027     - ETA: 0s - loss: 0.50
Epoch 5/30
1706/1706 [==============================] - 2s - loss: 0.4916     
Epoch 6/30
1706/1706 [==============================] - 2s - loss: 0.4780     - ETA: 2s - loss:  - ETA: 1s - l
Epoch 7/30
1706/1706 [==============================] - 2s - loss: 0.4643     
Epoch 8/30
1706/1706 [==============================] - 2s - loss: 0.4524     
Epoch 9/30
1706/1706 [==============================] - 2s - loss: 0.4413     
Epoch 10/30
1706/1706 [==============================] - 2s - loss: 0.4426     
Epoch 11/30
1706/1706 [==============================] - 2s - loss: 0.4331     
Epoch 12/30
1706/1706 [==============================] - 3s - loss: 0.4310 